In [ ]:
# 읽어올 파일 이름
file_name_train = "train_datas.csv"
file_name_test  = "test_datas.csv"

In [ ]:
import pandas as pd
# 학습 데이터, 훈련 데이터 파일 불러오기
train_data = pd.read_csv(file_name_train, encoding='utf-8-sig')
test_data = pd.read_csv(file_name_test, encoding='utf-8-sig')

In [12]:
print(train_data.head())
print(test_data.head())

                                                text  label  predicted_label  \
0                                       가격 대비 만족합니다.      1                1   
1                                          잘 쉬다 갑니다.      1                1   
2                   전주 시내에서 가까운 모텔가에 인접해 있고 찾기 쉽습니다.      1                1   
3  그리고 웬만한 수도권 호텔 못지않게 인테리어, 청결도, 서비스까지 깔끔하고 완벽했습니다.      1                1   
4  물 수압도 세고 바닥과 벽면이 금방 제습되어 습하지 않고, 무엇보다 에어컨에서 담배...      1                0   

   positive_prob  
0       0.999516  
1       0.999371  
2       0.952101  
3       0.999582  
4       0.839854  
                                        text  label  predicted_label  \
0                         전북대병원과 가까워 방문했습니다.      1                1   
1  도로 옆이 아니라 조용하고 생각보다 깔끔하여 하룻밤 묵기 좋은 위치입니다.      1                1   
2     매우 예쁜 방이라 사진을 많이 찍고 싶었지만 시간이 너무 늦었습니다.      1                1   
3                        그래도 따뜻하게 잘 묵고 왔습니다.      1                1   
4       방이 아늑하고 햇빛 차단이 잘 되어 알람도 못 듣고 푹 잤습니다. 

In [13]:
# 훈련용 리뷰 데이터 개수 출력
print(f"훈련용 리뷰 개수 : {len(train_data)}")
# 테스트용 리뷰 데이터 개수 출력
print(f"테스트용 리뷰 개수 : {len(test_data)}")

훈련용 리뷰 개수 : 42141
테스트용 리뷰 개수 : 10315


In [14]:
train_data['predicted_label'].value_counts()

predicted_label
0    23433
1    18708
Name: count, dtype: int64

In [15]:
test_data['predicted_label'].value_counts()

predicted_label
0    5508
1    4807
Name: count, dtype: int64

In [16]:
# 중복 제거
train_data.drop_duplicates(subset=['text'], inplace=True)
test_data.drop_duplicates(subset=['text'], inplace=True)
# y_review.drop_duplicates(subset=['review_content'], inplace=True)

In [17]:
import numpy as np
# 결측치 제거
train_data['text'] = train_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
train_data['text'] = train_data['text'].str.replace('^ +','',regex=True)
train_data.dropna(how='any', inplace=True)

In [18]:
test_data['text'] = test_data['text'].str.replace('[^a-zA-Z가-힣0-9 ]','',regex=True)
test_data['text'] = test_data['text'].str.replace('^ +','',regex=True)
test_data.dropna(how='any', inplace=True)

In [19]:
# 중복, 결측치 제거 후 데이터 개수
print(f"전처리 후 훈련 데이터 개수 : {train_data.shape}")
print(f"전처리 후 테스트 데이터 개수 : {test_data.shape}")
# print(f"전처리 후 테스트 데이터 개수 : {y_review.shape}")

전처리 후 훈련 데이터 개수 : (37385, 4)
전처리 후 테스트 데이터 개수 : (9499, 4)


In [20]:
# 불용어 목록 선언
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다', '용하다', '물이',
    '뿐', '대로', '만', '따름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [21]:
# 형태소 분석기 객체 생성
from konlpy.tag import Okt
okt  = Okt()

In [22]:
# 작업의 진행 상황을 시각적으로 표시해주는 라이브러리
from tqdm import tqdm

In [24]:
X_train = []
y_train = []

for sentence, label in tqdm(zip(train_data['text'], train_data['predicted_label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_train.append(filtered)
        y_train.append(label)

37385it [01:28, 421.61it/s]


In [25]:
print(X_train[:20])
print(y_train[:20])

[['가격', '대비', '만족합니다'], ['잘', '쉬', '갑니다'], ['전주', '시내', '가까운', '모텔', '가에', '인접', '해', '있고', '찾기', '쉽습니다'], ['웬만한', '수도권', '호텔', '못지않게', '인테리어', '청결', '도', '서비스', '깔끔하고', '완벽했습니다'], ['물', '수압', '도', '세고', '바닥', '벽면', '금방', '제', '습', '습하지', '않고', '에어컨', '담배', '냄새', '나지', '않습니다'], ['오래된', '곳', '거의', '담배', '냄새', '배어있는데', '다시', '전주', '방문', '하게', '되면', '재', '방문', '할'], ['여행', '기간', '중', '사용', '했던', '게스트하우스', '중', '제일', '좋았습니다'], ['위치', '가성', '비는', '말', '할', '도', '없었으며', '야', '놀자', '리뷰', '보고', '걱정', '했었는데', '정말', '잘', '쉬', '갑니다'], ['속', '터미널', '가까워', '이동', '편리했습니다'], ['방도', '깨끗하고', '난방', '잘', '추운', '날씨', '에도', '따뜻하게', '잘', '보냈습니다'], ['다음', '전주', '오게', '되면', '다시', '이용', '싶습니다'], ['터미널', '가까워', '좋았지만', '화장실', '전기', '들어오고', '휴대폰', '충전기', '도', '고장', '나서', '숙박', '불편했습니다'], ['넷플릭스', '방', '결제', '했는데', '넷플릭스', '방', '돈', '더', '줘야', '한다고', '했습니다'], ['처음', '결제', '할', '가격', '다르게', '하셨으면', '가격', '결제', '했을'], ['침대', '스프링', '꺼져', '있습니다'], ['아주', '좋습니다'], ['매우', '좋습니다'], ['베개', '꿉', '꿉한', '냄새', '났습니다'], ['사장

In [26]:
X_test = []
y_test = []

for sentence, label in tqdm(zip(test_data['text'], test_data['predicted_label'])):
    tokens = okt.morphs(sentence)
    filtered = [w for w in tokens if w not in stopwords]
    if len(filtered) > 0:
        X_test.append(filtered)
        y_test.append(label)

9499it [00:22, 423.86it/s]


In [28]:
y_train = np.array(train_data['predicted_label'], dtype=int)
y_test = np.array(test_data['predicted_label'], dtype=int)

In [29]:
import pickle
with open('new_X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('new_X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f)
with open('new_y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)
with open('new_y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [30]:
print(X_train[:10])
print(X_test[:10])
print(y_train[:10])
print(y_test[:10])

[['가격', '대비', '만족합니다'], ['잘', '쉬', '갑니다'], ['전주', '시내', '가까운', '모텔', '가에', '인접', '해', '있고', '찾기', '쉽습니다'], ['웬만한', '수도권', '호텔', '못지않게', '인테리어', '청결', '도', '서비스', '깔끔하고', '완벽했습니다'], ['물', '수압', '도', '세고', '바닥', '벽면', '금방', '제', '습', '습하지', '않고', '에어컨', '담배', '냄새', '나지', '않습니다'], ['오래된', '곳', '거의', '담배', '냄새', '배어있는데', '다시', '전주', '방문', '하게', '되면', '재', '방문', '할'], ['여행', '기간', '중', '사용', '했던', '게스트하우스', '중', '제일', '좋았습니다'], ['위치', '가성', '비는', '말', '할', '도', '없었으며', '야', '놀자', '리뷰', '보고', '걱정', '했었는데', '정말', '잘', '쉬', '갑니다'], ['속', '터미널', '가까워', '이동', '편리했습니다'], ['방도', '깨끗하고', '난방', '잘', '추운', '날씨', '에도', '따뜻하게', '잘', '보냈습니다']]
[['전북대', '병원', '가까워', '방문', '했습니다'], ['도로', '옆', '아니라', '조용하고', '생각', '깔끔하여', '하룻밤', '묵기', '좋은', '위치'], ['매우', '예쁜', '방이', '라', '사진', '많이', '찍고', '싶었지만', '시간', '너무', '늦었습니다'], ['그래도', '따뜻하게', '잘', '묵고', '왔습니다'], ['방이', '아늑하고', '햇빛', '차단', '잘', '알람', '도', '못', '듣고', '푹', '잤습니다'], ['다음', '에는', '바비큐', '도', '해보고', '싶습니다'], ['불', '켜지지', '않는', '곳', '많았고', '충전기', '도', '작